# 第５章　誤差逆伝搬法

第4章では数値微分と勾配法を使って、ニューラルネットワークの学習が可能であることを示しました。実際の学習では計算時間および計算精度の観点から数値微分は使われずに、ニューラルネットワークの解析的微分を効率よく計算するためのアルゴリズムである誤差逆伝搬法が使われます。この章では誤差逆伝搬法を学習します。

## 単純な関数の解析的微分

単純な関数の微分は解析的な表現が知られていることが多いです。

$$
f(x) = x^n \Rightarrow \frac{df}{dx} = nx^{n-1} \\
f(x) = \sin(ax) \Rightarrow \frac{df}{dx} = a\cos(ax) \\
f(x) = \cos(ax) \Rightarrow \frac{df}{dx} = -a\sin(ax) \\
f(x) = \exp(ax) \Rightarrow \frac{df}{dx} = a\exp(ax)
$$


## 合成関数の微分

では、複雑な関数の微分はどのように計算するのでしょうか。ここで二つの関数
$$
y = f(x) \\
z = g(y)
$$
が与えられていると、その合成関数$g\circ f$を考えます。
$$
(g\circ f)(x) = g(f(x))
$$

多くの複雑な関数は単純な関数の合成で表現することができることに注意してください。
実は、合成関数$g\circ f$の微分は次のように、部品となった関数$f, g$の微分で表現することができます。

$$
    \frac{dz}{dx} = \frac{dz}{dy} \frac{dy}{dx}
$$

変数が複数ある場合は偏微分を使った次のような拡張が必要になります。例えば次のような関数を考えます。
$$
y_1 = f_1(x_1, x_2) \\
y_2 = f_2(x_1, x_2) \\
z = g(y_1, y_2)
$$
ここで、偏微分は次のように計算できることが知られています。
$$
\frac{\partial z}{\partial x_1} = \frac{\partial z}{\partial y_1}\frac{\partial y_1}{\partial x_1} + \frac{\partial z}{\partial y_2}\frac{\partial y_2}{\partial x_1}
$$
一般化して、ベクトルで表現すると、
$$
    {\bf y} = {\bf f}({\bf x}) \\
    z = g({\bf y})
$$
に対して、
$$
    \frac{\partial z}{\partial x_1} = \frac{\partial z}{\partial {\bf y}} \cdot \frac{\partial {\bf y}}{\partial x_1}
$$
と表現される

# ニューラルネットワークの場合

ニューラルネットワークは行列積と活性化層による非線形変換の組み合わせで表現されます。例えばニューラルネットワークの単一層は次のように表現されます。
$$
    {\bf a} = {\bf W}{\bf x} + {\bf B} \\
    {\bf z} = {\bf h}({\bf a})
$$
出力${\bf z}$の重みに対する微分は次のように表現できます。
$$
    \frac{\partial {\bf z}}{\partial{\bf W}} 
    = \frac{\partial {\bf z}}{\partial{\bf a}}\cdot \frac{\partial {\bf a}}{\partial{\bf W}} \\
    = {\bf h}'({\bf a})\cdot {\bf x}
$$

## 単純なレイヤの実装

計算グラフの乗算ノードを「乗算レイヤ(MulLayer)」、加算ノードを「加算レイヤ(AddLayer)」として実装する。

### 乗算レイヤの実装

In [9]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        dx = dout * self.y # xとyをひっくり返す
        dy = dout * self.x
        return dx, dy

In [10]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price) # 220

220.00000000000003


### 加算レイヤの実装

In [11]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [12]:
apple = 100
apple_num = 2 
orange = 150 
orange_num = 3 
tax = 1.1

# layer
mul_apple_layer = MulLayer() 
mul_orange_layer = MulLayer() 
add_apple_orange_layer = AddLayer() 
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) #(3) 
price = mul_tax_layer.forward(all_price, tax) #(4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) #(3) 
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)
print(price) # 715
print(dapple_num, dapple, dorange, dorange_num, dtax) # 110 2.2 3.3 165 650

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 活性化関数レイヤの実装

### ReLU レイヤ

ReLU(Rectified Linear Unit)
$$
\begin{eqnarray}
\left\{
y = 
\begin{array}{l}
x (x>0) \\
0 (x\leqq0)
\end{array}
\right.
\end{eqnarray}
$$

上記の$x$に関する$y$の微分
$$
\begin{eqnarray}
\left\{
\frac{\partial y}{\partial x} = 
\begin{array}{l}
1 (x>0) \\
0 (x\leqq0)
\end{array}
\right.
\end{eqnarray}
$$


In [13]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

### Sigmoid レイヤ

Sigmoid関数

$$
y = \frac{1}{1+{\rm exp}(-x)}
$$

計算グラフ

1. $-x$ ..($y_1=-x$)

2. ${{\rm exp}(-x)}$ ..($y_2={\rm exp}(y_1)$)

3. $1+{{\rm exp}(-x)}$ ..($y_3={1+y_2}$)

4. $\frac{1}{1+{{\rm exp}(-x)}}$ ..($y=\frac{1}{y_3}$)

逆伝搬の流れ

4.->3.->2.->1. のように$y$から$x$へ逆順で微分値を伝搬していく

$$
\frac{\partial y}{\partial x} = \frac{\partial y}{\partial y_3}\frac{\partial y_3}{\partial y_2}\frac{\partial y_2}{\partial y_1}\frac{\partial y_1}{\partial x}
$$

4. の微分

$$
\begin{eqnarray}
\frac{\partial y}{\partial y_3}
&=& \frac{\partial}{\partial y_3} \frac{1}{y_3} \\
&=& -\frac{1}{y_3^2} \\
&=& -y^2
\end{eqnarray}
$$


3. の微分

$$
\begin{eqnarray}
\frac{\partial y_3}{\partial y_2}
&=& \frac{\partial}{\partial y_2}1+y_2 \\
&=& 1
\end{eqnarray}
$$
$$
\frac{\partial y}{\partial y_2} = -y^2 
$$

2. の微分

$$
\begin{eqnarray}
\frac{\partial y_2}{\partial y_1}
&=& \frac{\partial}{\partial y_1} {\rm exp}(y_1) \\
&=& {\rm exp}(y_1) \\
&=& {\rm exp}(-x)
\end{eqnarray}
$$
$$
\begin{eqnarray}
\frac{\partial y}{\partial y_1} = -y^2 {\rm exp}(-x)
\end{eqnarray}
$$



1. の微分

$$
\begin{eqnarray}
\frac{\partial y_1}{\partial 1}
&=& \frac{\partial}{\partial x} -x \\
&=& -1
\end{eqnarray}
$$
$$
\frac{\partial y}{\partial x} = y^2 {\rm exp}(-x)
$$

これは下記のように整理できる

$$
\begin{eqnarray}
y^2 {\rm exp}(-x) 
&=& \frac{1}{(1 + {\rm exp}(-x))^2}{\rm exp}(-x) \\
&=& \frac{1}{1 + {\rm exp}(-x)}\frac{{\rm exp}(-x)}{1 + {\rm exp} (-x)} \\
&=& y(1 - y)
\end{eqnarray}
$$

整理の結果$x$を使わなくてよくなった

Sigmoid関数の逆伝搬値は、順伝搬の出力結果$y$だけから計算することができる

順伝搬： $x$ → sigmoid → $y$

逆伝搬： $\frac{\partial L}{\partial y}y(1-y)$ ← sigmoid ← $\frac{\partial L}{\partial y}$

$  \frac{\partial L}{\partial x} = \frac{\partial L}{\partial y}y(1-y) $


In [14]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
        
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## 5.6 Affine ／ Softmax レイヤの実装

### 5.6.1 Affine レイヤ




In [15]:
import numpy as np
X = np.random.rand(2) # 入力
W = np.random.rand(2,3) # 重み
B = np.random.rand(3) # バイアス

X.shape # (2,)
W.shape # (2, 3)
B.shape # (3,)

Y = np.dot(X, W) + B
print(X)
print(W)
print(B)
print(Y)

[0.82824056 0.40601607]
[[0.76025466 0.16582513 0.12144279]
 [0.97198465 0.36913364 0.91313187]]
[0.99180796 0.67971729 0.20091332]
[2.0161231  0.96693458 0.67224338]


グラフ

1. $\bf{X} \cdot \bf{W}$

2. $(\bf{X} \cdot \bf{W}) + \bf{B}$

逆伝搬

$\frac{\partial L}{\partial \bf{X}} = \frac{\partial L}{\partial \bf{Y}} \cdot \bf{W}^T $

$\frac{\partial L}{\partial \bf{X}} = (\frac{\partial L}{\partial x_0}, \frac{\partial L}{\partial x_1}, ...\frac{\partial L}{\partial x_n}) $ 

$\frac{\partial L}{\partial \bf{W}} = \bf{X}^T \cdot \frac{\partial L}{\partial \bf{Y}} $ 

$\frac{\partial L}{\partial \bf{B}} = \frac{\partial L}{\partial \bf{Y}} $ の最初の軸に関する和 



In [16]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx